Assignment - Scraping Taoiseach wikipedia webpage

### 1. Provide the commands required to create a new MariaDB database to store your scraped data. You are to further provide the commands which you used to define a new user-id and password to the database, as well the appropriate table schema needed to house your scraped web data.

sudo mysql

create database TaoiseachDB;
grant all on TaoiseachDB.* to 'Tuser' identified by 'Tpasswd';
flush privileges;
quit
--Login to the TaoiseachDB
mysql -u Tuser -p TaoiseachDB

Create table Taoiseach(
Name varchar(64),
Constituency1 varchar(64),
Constituency2 varchar(64),
Term_Of_Office_Start date,
Term_Of_Office_End date,
Party varchar(64),
Vice_President varchar(64),
Dail int,
Date_Of_Birth date)

select * from Taoiseach;
Describe Taoiseach;

### 2. Write Python code which automatically scrapes the data you need from the above Wikipedia page and stores it into your database table(s).

In [ ]:
import pandas as pd
import numpy as np
from pandas.io.html import read_html

In [ ]:
page = "https://en.wikipedia.org/wiki/Taoiseach"
tables = read_html(page, attrs = {'class':'wikitable'})
print(" {num} tables".format(num = len(tables)))

In [ ]:
tables[0]

In [ ]:
heading = tables[0].loc[1]
heading

In [ ]:
df = tables[0]
df = df.loc[2:48]
df.columns = heading
df

In [ ]:
df = df.drop("No.", axis = "columns")
df = df.drop("Portrait", axis = "columns")
df.columns = ["Name", "Term_of_Office_Start","Term_of_office_End","Party","Exec_CouncilComposition",
              "Not_Required","Not_Required","Vice_President","Dail"]
df = df.drop("Not_Required", axis = "columns")
df

In [ ]:
df1 =  df["Name"].str.split('(', expand = True)[1]
Name1 = df["Name"].str.split('(', expand = True)[0]
df1.str.split('for ', expand = True)[1]
Constituency1 = df1.str.split('for ', expand = True)[1]
Constituency2 = df1.str.split('for ', expand = True)[2]
Term_Of_Office_Start = df["Term_of_Office_Start"].str.split('[', expand = True)[0]
Term_Of_Office_End = df["Term_of_office_End"].str.split('[', expand = True)[0]
Dail_Elected = df["Dail"].str.split('(', expand = True)[0]
df = pd.concat([df, Name1, Constituency1, Constituency2,
           Term_Of_Office_Start, Term_Of_Office_End, Dail_Elected ], axis = 1)
df

In [ ]:
df = df.drop("Name", axis = "columns")
df = df.drop("Term_of_Office_Start", axis = 1)
df = df.drop("Term_of_office_End", axis = 1)
df = df.drop("Dail", axis = 1)
df = df.drop("Exec_CouncilComposition", axis = 1)
df

In [ ]:
df.columns = ["Party", "Vice_President", "Name",
             "Constituency1", "Constituency2", "Term_Of_Office_Start", "Term_Of_Office_End", "Dail"]
df = df.drop([10,11])
df["Constituency1"] = df["Constituency1"].str.split("until", expand = True)[0]
df["Constituency2"] = df["Constituency2"].str.split("from", expand = True)[0]
df["Term_Of_Office_Start"] = pd.to_datetime(df["Term_Of_Office_Start"], errors="coerce")
df["Term_Of_Office_End"] = pd.to_datetime(df["Term_Of_Office_End"], errors="coerce")
df = df[["Name", "Constituency1", "Constituency2", "Term_Of_Office_Start", "Term_Of_Office_End",
   "Party", "Vice_President", "Dail"]]
df

In [ ]:
df.dtypes

In [ ]:
Url_Initial = "https://en.wikipedia.org/wiki/"
Url_Person = df["Name"]
Url_Total = Url_Initial+Url_Person
len(Url_Total)

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
DOB_List = []
def scrape_DOB(url):
    response = requests.get(url)
    r = response.text
    soup = BeautifulSoup(r,"lxml")
    h1 = soup.find(attrs={"class":"bday"})
    DOB_List.append(h1.text)
for url in Url_Total:
    scrape_DOB(url)
print(DOB_List) 

In [ ]:
col_dob = pd.DataFrame(DOB_List)
df = df.reset_index()
df = pd.concat([df, col_dob], axis=1)
df = df.drop("index", axis = 1)
df.columns = ["Name", "Constituency1", "Constituency2", "Term_Of_Office_Start", "Term_Of_Office_End",
   "Party", "Vice_President", "Dail", "Date_Of_Birth"]
df["Date_Of_Birth"] = pd.to_datetime(df["Date_Of_Birth"], errors="coerce")
df

In [ ]:
df_list = df.values.tolist()
df_list

In [ ]:
df.info()

In [ ]:
import DBcm
config = {
    'user': 'Tuser',
    'password': 'Tpasswd',
    'host': 'localhost',
    'database': 'TaoiseachDB',
}

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = "insert into Taoiseach (Name, Constituency1, Constituency2, Term_Of_Office_Start, Term_Of_Office_End, Party, Vice_President, Dail, Date_Of_Birth) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(SQL, df_list)

### Question-3

#### (a) Provide a list of constituencies in Ireland which have produced a Taoiseach.

select distinct Constituency1  as All_Constituencies from Taoiseach
union
select distinct Constituency2 from Taoiseach where Constituency2 is not null;

#### (b) Which political party has produced the most individual Taoiseachs?

Select count(distinct Name) as No_Of_President, Party from Taoiseach group by Party order by No_Of_President desc limit 1;

#### (c) Provide the list of names for those politicians who previously held the position of “Vice President” or “Tánaiste” prior to becoming Taoiseach.

select distinct Name from Taoiseach where Name in (select distinct Vice_President from Taoiseach);

#### (d) Which political party held the office of Taoiseach for the longest amount of overall time, and for how long (in years, months, and days)?

select sum(distinct datediff(Term_Of_Office_End, Term_Of_Office_Start)) as Total_Time_Days, Party
from Taoiseach group by Party order by Total_Time_Days desc limit 1;

#### (e) Which individual politician held the office of Taoiseach for the longest amount of uninterrupted time, and for how long (in years, months, and days)?

select date_format(FROM_DAYS(datediff(Term_Of_Office_End,Term_Of_Office_Start)), '%Y-%m-%d') as Days, Name 
from Taoiseach group by Name, Days order by Days desc limit 1;

#### (f) Which individual politician held the office of Taoiseach for the shortest amount of time, and for how long (in years, months, and days)?

select Name,datediff(Term_Of_Office_End, Term_Of_Office_Start) as Time_Days 
from Taoiseach 
order by Time_Days asc limit 1;

#### (g) Which political party has held the office of Taoiseach for the largest number of Dáils?

Select count(distinct Dail) as No_of_Dial, Party from Taoiseach group by Party order by No_of_Dial desc limit 1;

#### (h) Which individual politician has held the office of Taoiseach for the most number of Dáils?

Select  count( distinct Dail) as No_Of_Dails, Name from Taoiseach group by Name order by No_Of_Dails desc limit 1;

#### (i) What age was each Taoiseach on the day they assumed office?

select distinct Name, Floor(datediff(Term_Of_Office_Start, Date_Of_Birth)/365) as Age_Joined_Office from Taoiseach order by Age_Joined_Office;

#### (j) On the last day of their term of office, which politician was the oldest?

select distinct Name, date_format(FROM_DAYS(datediff(Term_Of_Office_End, Date_Of_Birth)), '%Y-%m-%d') as Age_YearMonthDays 
from Taoiseach order by Age_YearMonthDays Desc limit 1;

#### (k) In your view, and based on statistics calculated from your scraped data, which Irish political party is the most successful? Show and describe your calculations.

Through the queries below, we can see the following:

select count(distinct Name) No_Of_President_In_Party, Party from Taoiseach group by party order by No_Of_President_In_Party desc;
-- We can see that "Fianna Fáil" party had ruled with 7 Taoiseach till now. 

select count(distinct dail) No_Dails_Formed, Party from Taoiseach group by party order by No_Dails_Formed desc;
-- We can see that "Fianna Fail" party had 19 Dails formed in their ruling time. 

select count(distinct Vice_President) as No_Of_VicePresident,Party from Taoiseach group by Party order by No_Of_VicePresident desc;
-- We can see that "Fianna Fail" party had 15 Vice_President till now.

select sum(datediff(Term_Of_Office_End, Term_Of_Office_Start)) as Total_Time_Days, Party
from Taoiseach group by Party order by Total_Time_Days desc limit 1;
-- We can see that "Fianna Fail" party has rules over Ireland for the longest duration till now which is 
    61 years, 1 months and 13 days. So through the above data "Fianna Fail" party is most successfull.